<a href="https://colab.research.google.com/github/koyomin9zx/UITQA-Vietnamese-Question-Answering/blob/master/Relevant_Ranking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:

from google.colab import drive
drive.mount('/content/drive')

In [0]:
!ln -s /content/drive/'My Drive'/data_QA

In [0]:
!pip install underthesea 
!pip install unidecode

In [0]:
# importing libraries 
from underthesea import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.metrics.pairwise import cosine_similarity 
from scipy.spatial import distance 
from collections import defaultdict, OrderedDict 
from string import punctuation
import unidecode
import re
import pandas as pd 
import numpy as np 
import glob

In [0]:
def load_data(path):
  data=[]
  all_files = glob.glob(path + "/*.txt")
  for file in all_files:
      passage=open(file, "r", encoding='utf-8').read()
      data.append(passage)
  return data

def vi_tokenizer(row):
    return word_tokenize(row, format="text")

def remove_accents(row):
  return unidecode.unidecode(row)

def standardize_data(df):
    hl_cleansed=[]
    remove = punctuation
    #remove = remove.replace("-", "") # don't remove hyphens
    pattern = "[{}]".format(remove) # create the pattern
    re_space=re.compile('\s+')
    re_trailing=re.compile('^\s+|\s+?$')
    for row in df:
        row=re.sub(pattern, " ", row) 
        row=re.sub(re_space,' ',row)
        row=re.sub(re_trailing,' ',row)
        row = row.strip()
        row = vi_tokenizer(row)
        row = remove_accents(row)
        row = row.lower()
        hl_cleansed.append(row)
    return hl_cleansed

In [0]:
## Converting 3D array of array into 1D array 
def arr_convert_1d(arr): 
    arr = np.array(arr) 
    arr = np.concatenate( arr, axis=0 ) 
    arr = np.concatenate( arr, axis=0 ) 
    return arr 
   
## Cosine Similarity 
def cosine(trans): 
    cos = [] 
    cos.append(cosine_similarity(trans[0], trans[1])) 
    return cos

def tfidf(str1, str2,tf_idf_vetor):  
    corpus = [str1,str2] 
    trans = tf_idf_vetor.transform(corpus) 
    cos=cosine(trans) 
    return arr_convert_1d(cos)[0]
   

In [96]:
data=load_data('/content/data_QA')
vect = TfidfVectorizer(min_df=2, max_df=0.8,max_features=2000,sublinear_tf=True) 
vect.fit(data)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=0.8, max_features=2000,
                min_df=2, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=True, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [104]:
query='Chu Văn An sinh năm nào?'

score=defaultdict()
for i in data:
  t=tfidf(query, i,vect)
  score[t]=i


score=OrderedDict(sorted(score.items(),reverse=True))

print(standardize_data([query]))

['chu_van_an_sinh nam nao']
